In [26]:
# Initialize Otter
import otter
grader = otter.Notebook("hwk3-task1-spills.ipynb")

# Task 1: Visualizing oil spills in NY State

## Instructions

- First, update the following cell to have a link to *your* Homework 2 GitHub repository:

**UPDATE THIS LINK**
https://github.com/kaimorquecho/eds220-hwk3.git


- Review the [complete rubric for this task](https://docs.google.com/document/d/1x0BoU6IH4cnOR1-n7i9CYQ9wUC37yDpYlQ4j6rCfcsU/edit?tab=t.0) before starting.

- **Meaningful commits should be made every time you finish a major step.** We'll check your repository and view the commit history.

- Comment mindfully in a way that enriches your code. Comments should follow best practices.

- **Do not update the top cell with the `otter` import**, this is used internally for grading.


## About the data
In this task you will use two datsets:

### First dataset: oil spills data

The first dataset contains information about [spilling incidents in the state of New York](https://data.ny.gov/Energy-Environment/Spill-Incidents/u44d-k5fk). 
The data portal has plenty of information. 
You will find extra information and metadata for this datset in the `NYSDEC_SpillIncidents_Overview.pdf` and `NYSDEC_SpillIncidents_DataDictionary.pdf` documents available at the portal.

### Second dataset: TIGER shapefile

The second dataset is the [the 2023 Counties (and equivalent) TIGER shapefile](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2022.html#list-tab-790442341) from the Census Bureau. You can check the [metadata for all the TIGER shapefiles here](https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line.html). 

## 1. File management
a. Create a `data/` directory inside your `eds220-hwk3` directory. If you download any data, it should be transferred into this folder. 

b. Add the `data/` directory to your `.gitignore` file and push your changes. This is IMPORTANT: the datasets we'll be working with are large and they will most likely jam your commit history if you try to push them.

## 2. Datasets description
- Read the metadata for both datsets and write a brief description about each one. Indicate the date sources and date of access.

## 3. Import libraries

In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

## 4. Import data 
In the next cell:

- Import the NY spills data as a variable named `spills`. 
- Import the US counties data as a variable named `counties`.

If you are using file paths to access the data, these should be constructed using the `os` package.

In [28]:
os.environ['PROJ_LIB'] = '/opt/anaconda3/share/proj'


fp = os.path.join('data','Spill_Incidents_20251107.csv')
spills = pd.read_csv(fp)


fp = os.path.join('data','tl_2023_us_county','tl_2023_us_county.shp')
counties = gpd.read_file(fp)


<!-- BEGIN QUESTION -->

## 5. Prepare spills data

This section refers to the `spills` dataframe. 
Check the outputs carefully, they will give you information you will need to use in the next exercises.

### 5.A. Exploration

In the next cells:

a. Check the dataframe's head

b. Check the data types of the columns

c. Check the unique values in the `material_family` column

d. Check the unique values in the `county` column

e. Insert a markdown cell to explain what information you obtained from the preliminary exploration. Use complete sentences.

You can add any other cells for preliminary data exploration.

In [29]:
#a. Check the dataframe's head
spills.head(5)

,Spill Number,Program Facility Name,Street 1,Street 2,Locality,County,ZIP Code,SWIS Code,DEC Region,Spill Date,Received Date,Contributing Factor,Waterbody,Source,Close Date,Material Name,Material Family,Quantity,Units,Recovered
0,107132,MH 864,RT 119/MILLWOOD RD,NaN,ELMSFORD,Westchester,NaN,6000,3,10/10/2001,10/10/2001,Unknown,NaN,Unknown,10/15/2001,unknown material,Other,10,Gallons,0
1,405586,BOWRY BAY,WATER POLL CONTROL,NaN,QUEENS,Queens,NaN,4101,2,08/21/2004,08/21/2004,Other,EAST RIVER,Unknown,09/17/2004,raw sewage,Other,0,NaN,0
2,405586,BOWRY BAY,WATER POLL CONTROL,NaN,QUEENS,Queens,NaN,4101,2,08/21/2004,08/21/2004,Other,EAST RIVER,Unknown,09/17/2004,raw sewage,Other,0,Pounds,0
3,204667,POLE 16091,GRACE AVE/BURKE AVE,NaN,BRONX,Bronx,NaN,301,2,08/02/2002,08/02/2002,Equipment Failure,NaN,Commercial/Industrial,10/28/2002,transformer oil,Petroleum,1,Gallons,0
4,210559,POLE ON,FERDALE LOMIS RD / RT 52,NaN,LIBERTY,Sullivan,NaN,5336,3,01/20/2003,01/20/2003,Traffic Accident,NaN,Commercial/Industrial,01/22/2003,transformer oil,Petroleum,6,Gallons,6


In [30]:
#b. Check the data types of the columns
spills.dtypes

Spill Number              int64
Program Facility Name    object
Street 1                 object
Street 2                 object
Locality                 object
County                   object
ZIP Code                 object
SWIS Code                 int64
DEC Region                int64
Spill Date               object
Received Date            object
Contributing Factor      object
Waterbody                object
Source                   object
Close Date               object
Material Name            object
Material Family          object
Quantity                 object
Units                    object
Recovered                object
dtype: object

In [31]:
#c. Check the unique values in the `material_family` column

# Simplify column names in spills DataFrame
spills.columns = spills.columns.str.lower().str.replace(' ','_') 

# View new column names
print('Simplified column names:\n', spills.columns.tolist())

# View material_family unique values
print('\nmaterial_family unique values:\n',spills['material_family'].unique()) # Output: array of unique values in 'material_family' column

Simplified column names:
 ['spill_number', 'program_facility_name', 'street_1', 'street_2', 'locality', 'county', 'zip_code', 'swis_code', 'dec_region', 'spill_date', 'received_date', 'contributing_factor', 'waterbody', 'source', 'close_date', 'material_name', 'material_family', 'quantity', 'units', 'recovered']

material_family unique values:
 ['Other' 'Petroleum' 'Hazardous Material' 'Oxygenates']


In [32]:
#d. Check the unique values in the `county` column
spills['county'].unique()

array(['Westchester', 'Queens', 'Bronx', 'Sullivan', 'Cortland',
       'New York', 'Ulster', 'Kings', 'Orange', 'Dutchess', 'Onondaga',
       'Saratoga', 'Cayuga', 'Oswego', 'Warren', 'Niagara', 'Rockland',
       'Nassau', 'Jefferson', 'Schenectady', 'Albany', 'Monroe',
       'Schuyler', 'St Lawrence', 'Richmond', 'Clinton', 'Lewis', 'Essex',
       'Chenango', 'Erie', 'Livingston', 'Oneida', 'Wayne', 'Suffolk',
       'Orleans', 'Ontario', 'Genesee', 'Otsego', 'Tompkins', 'Madison',
       'Chemung', 'Seneca', 'Broome', 'Hamilton', 'Washington', 'Steuben',
       'Rensselaer', 'Franklin', 'Columbia', 'Fulton', 'Herkimer',
       'Schoharie', 'Montgomery', 'Putnam', 'Delaware',
       'New Jersey - Region 2', 'Tioga', 'Chautauqua', 'Cattaraugus',
       'Wyoming', 'Yates', 'Greene', 'Pennsylvania - Region 9',
       'Allegany', 'New Jersey - Region 3 (N)',
       'Cattaraugus Indian Reservation', 'New Jersey - Region 3 (T)',
       'Canada - Region 6', 'Canada - Region 9',
       '

e. Type your answer here, replacing this text.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 5.B. Cleaning
a. Simplify column names as needed.

I simplified column names in cell C since instructions asked to check column 'material_family' and the column was not yet named that

```
spills.columns = spills.columns.str.lower().str.replace(' ','_')
```

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

b. Update column data types as needed *for this analysis*.

In [33]:
# Update 'spill_date' to a datetime object 
spills['spill_date'] = pd.to_datetime(spills['spill_date'])

# Check that 'spill_date' is now datetime object
spills.dtypes

# Update index to column 'spill_date' and sort it
spills = spills.set_index('spill_date')

spills = spills.sort_index()


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 5.C. Data selection

a. Select data about petroleum spills that took place between January 1st 2023, and October 31st (including both dates), 2023. Store it in a variable named `petr_23`. The data selection should occur in a single line. You may add cells as needed.

In [34]:
spills.head(200)

,spill_number,program_facility_name,street_1,street_2,locality,county,zip_code,swis_code,dec_region,received_date,contributing_factor,waterbody,source,close_date,material_name,material_family,quantity,units,recovered
spill_date,,,,,,,,,,,,,,,,,,,
1900-01-01,2300485,BUS GARAGE,3867 NY-196,NaN,HARTFORD,Washington,NaN,5838,5,04/18/2023,Unknown,NaN,Unknown,01/15/2025,unknown petroleum,Petroleum,0,NaN,0
1900-01-01,9108142,REPUBLIC STEEL DUMP HUDSON RIVER (WATER ST?),HUDSON RIVER NEAR WWTP (MAIN ST? WATER ST?),REPUBLIC STEEL DUMP NEAR RCSD TREATMENT,TROY,Rensselaer,NaN,4217,4,10/04/1991,Other,GROUNDWATER HUDSON $,Commercial/Industrial,11/05/1991,coal tar pitch volatiles,Hazardous Material,0,Pounds,0
1900-01-22,2208599,UNKNOWN PETROLEUM ONTO MTA TRACKS,640 CONDUIT BLVD,NaN,BROOKLYN,Kings,NaN,2401,2,01/22/2023,Unknown,NaN,Unknown,01/23/2023,unknown petroleum,Petroleum,0,NaN,0
1900-12-29,9010299,695 9TH AVE/MANH,695 9TH AVENUE,NaN,MANHATTAN,New York,NaN,3101,2,12/22/1990,Tank Overfill,NaN,Commercial/Industrial,11/16/1994,#2 fuel oil,Petroleum,2,Gallons,0
1909-11-12,909514,ROADSIDE DITCH,4510 EAST RAIL ROAD,NaN,STOCKTON,Chautauqua,NaN,768,9,11/25/2009,Human Error,BEAR CREEK,Passenger Vehicle,12/07/2009,transmission fluid,Petroleum,2,Gallons,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975-03-19,7580320,MONROE FORGING,MCKEE RD.,NaN,ROCHESTER,Monroe,NaN,2814,8,03/19/1975,Tank Failure,BARGE CANAL,"Institutional, Educational, Gov., Other",05/14/1975,waste oil/used oil,Petroleum,0,Gallons,0
1975-03-19,7580324,MONROE FORGING/BARGE CANAL,MCKEE ROAD,NaN,GATES,Monroe,NaN,2826,8,NaN,Unknown,NaN,Unknown,02/17/2011,#2 fuel oil,Petroleum,"2,000",Gallons,0
1975-03-19,7580321,"ARTHUR V. TOWNER , INC.",1909 SCOTTSVILLE RD.,NaN,CHILI,Monroe,14623,2822,8,03/19/1975,Other,GENESEE RIVER,Unknown,03/19/1975,unknown material,Other,0,Gallons,0


KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

b. Use `print()` and f-strings to print a message verifying that `petr_23` only has data for 2023 and from January through October.
HINT: if you converted the `spill_date` column to datetime, you can use `petr_23.spill_date.dt.year.unique()`.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

c. Write tests using `assert` to verify that `petr_23` only has data for petroleum spills.

<!-- END QUESTION -->

### 5.D. Data aggregation

Create new dataframe named `spills_per_county` with the number of petroleum spills per county from January to October 2023 (i.e. use the `petr_23` dataframe).

The resulting `spills_per_county` dataframe must be as follows:

- Index: integer numbers starting from 0
- Two columns only.
- Column one: county names, column name = `county`
- Column two: number of petroleum spills per county in 2023, column name = `n_spills`

You may add cells if needed.

In [10]:
spills_per_county = ...
spills_per_county = ...

In [11]:
grader.check("q5d")

q5d results:
    q5d - 1 result:
        ❌ Test case failed
        Trying:
            def test_q3(student_data):
                try:
                    expected_data = pd.read_csv('data/q5d_df.csv').drop(columns='Unnamed: 0')
                    pd.testing.assert_frame_equal(expected_data, student_data)
                except AssertionError:
                    raise AssertionError('Incorrect answer.')
        Expecting nothing
        ok
        Trying:
            test_q3(spills_per_county)
        Expecting nothing
        **********************************************************************
        Line 7, in q5d 0
        Failed example:
            test_q3(spills_per_county)
        Exception raised:
            Traceback (most recent call last):
              File "<doctest q5d 0[0]>", line 4, in test_q3
                pd.testing.assert_frame_equal(expected_data, student_data)
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/site-packages/pandas/_testing/asserters.py", line 1210, in assert_frame_equal
                _check_isinstance(left, right, DataFrame)
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/site-packages/pandas/_testing/asserters.py", line 171, in _check_isinstance
                raise AssertionError(
            AssertionError: DataFrame Expected type <class 'pandas.core.frame.DataFrame'>, found <class 'ellipsis'> instead

            During handling of the above exception, another exception occurred:

            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q5d 0[1]>", line 1, in <module>
                test_q3(spills_per_county)
              File "<doctest q5d 0[0]>", line 6, in test_q3
                raise AssertionError('Incorrect answer.')
            AssertionError: Incorrect answer.

<!-- BEGIN QUESTION -->

## 6. Prepare counties data

This section refers to the `counties` geo-dataframe. 
Check the outputs carefully, they will give you context about the next exercises.

### 6.A. Exploration

In the next cells:

a. Check the geo-dataframe's head

b. Check the data types of the columns

c. Check the geo-dataframe's CRS

d. Plot the geo-dataframe

e. Use the next markdown cell to explain what information you obtained from the preliminary exploration. Use complete sentences.

You can add any other cells of preliminary data exploration.

_Type your answer here, replacing this text._

In [12]:
# a

In [13]:
# b

In [14]:
# c

In [15]:
# d

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 6.B. Cleaning

Simplify column names as needed.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 6.C. Data selection

Select all the counties for the state of New York in a new variable `ny_counties` and plot them (you may need to look at the metadata).

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 7 Merging

The goal of this section is to create a single dataframe with the number of spills per county and the corresponding geometries for each county.

### 7.A. Explore data

In the next cells:

a. Print the names of counties in `ny_counties`, without repetition.


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

b. Print the names of counties in `spills_per_county`, without repetition. Examine the differences in the outputs.

<!-- END QUESTION -->



c. Which are the county names that are in `spills_per_county`, but not in `ny_counties`? Your answer should be a list of strings
HINT: You can use the `numpy` function [`setdiff1d()`](https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html), pay attention to the example in the documentation. Store your answer in the `diff_names` variable. 


In [16]:
diff_names_spills = ...
diff_names_spills


Ellipsis

In [17]:
grader.check("7Ac")

7Ac results:
    7Ac - 1 result:
        ❌ Test case failed
        Trying:
            assert len(diff_names_spills) == 3
        Expecting nothing
        **********************************************************************
        Line 1, in 7Ac 0
        Failed example:
            assert len(diff_names_spills) == 3
        Exception raised:
            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest 7Ac 0[0]>", line 1, in <module>
                assert len(diff_names_spills) == 3
                       ^^^^^^^^^^^^^^^^^^^^^^
            TypeError: object of type 'ellipsis' has no len()

    7Ac - 2 result:
        ❌ Test case failed
        Trying:
            assert isinstance(diff_names_spills, list)
        Expecting nothing
        **********************************************************************
        Line 1, in 7Ac 1
        Failed example:
            assert isinstance(diff_names_spills, list)
        Exception raised:
            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest 7Ac 1[0]>", line 1, in <module>
                assert isinstance(diff_names_spills, list)
            AssertionError

d. Which are the county names that are in `ny_counties`, but not in `spills_per_county`? Your answer should be a list of strings.


In [18]:
diff_names_ny = ...
diff_names_ny

Ellipsis

In [19]:
grader.check("7Ad")

7Ad results:
    7Ad - 1 result:
        ❌ Test case failed
        Trying:
            assert len(diff_names_ny) == 1
        Expecting nothing
        **********************************************************************
        Line 1, in 7Ad 0
        Failed example:
            assert len(diff_names_ny) == 1
        Exception raised:
            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest 7Ad 0[0]>", line 1, in <module>
                assert len(diff_names_ny) == 1
                       ^^^^^^^^^^^^^^^^^^
            TypeError: object of type 'ellipsis' has no len()

    7Ad - 2 result:
        ❌ Test case failed
        Trying:
            assert isinstance(diff_names_ny, list)
        Expecting nothing
        **********************************************************************
        Line 1, in 7Ad 1
        Failed example:
            assert isinstance(diff_names_ny, list)
        Exception raised:
            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest 7Ad 1[0]>", line 1, in <module>
                assert isinstance(diff_names_ny, list)
            AssertionError

e. What is the number of spills in 'Tuscarora Indian Reservation'?


In [20]:
# e. 

### 7.B. Data updates

In the next cell, update 'St Lawrence' to 'St. Lawrence' in `spills_per_county` data frame and Verify the update was successful.


### 7.C. Merge

In the following cells:

a. Use this cell to make any other updates needed for making an inner join of the `spills_per_county` and `ny_counties`. 


b. Over-write `ny_counties` as the  inner join of the `spills_per_county` and `ny_counties` dataframes.



From our previous exploration we know that Tuscarora Indian Reservation will not be in the join. Tuscarora Indian Reservation is located within Niagara county. 

c. Since the Tuscarora Indian Reservation does not appear in the `ny_counties` dataframe, add one spill to Niagara county. We will add a note about this in our final map. Add the cells you need to make this update **and** verify that the dataframe was updated.

## 8. Map
a. Create a choropleth map of the number of petroleum spills per county in the state of New York in 2023. 
Your map should be polished and have (at least) the following updates:
- a legend indicating the number of spills
- an updated color map
- a title 
- no axis
- an annotation with the data source (New York State Department of Environmental Conservation) and date of access



b. Write a description of how the map was created, including key takeaways. If you need to do additional data analysis to include relevant information from the data, you may include additional cells here.

*Your answer here.*

## Complete workflow

**STILL WORKING ON THIS**

Collect all the relevant code into a **new notebook** titled `hwk3-task1-spills-COMPLETE-YOURLASTNAME.ipynb`. This notebook should have two sections:the a single cell will have the end-to-end workflow: from importing libraries and loading the data, to producing the graph. The *only* ouput of this cell should be the graph you produced in the previous exercise. Further guidance on what to include in this final workflow is in the assignment rubric.



**From last HW:**

Collect all the relevant code into the first blank cell of the notebook titled "COMPLETE WORKFLOW". This single cell will have the end-to-end workflow: from importing libraries and loading the data, to producing the graph. The only ouput of this cell should be the graph you produced in the previous exercise. Further guidance on what to include in this final workflow is in the assignment rubric.

##### Run the cell below to run all autograder tests. 

In [21]:
grader.check_all()

7Ac results:
    7Ac - 1 result:
        ❌ Test case failed
        Trying:
            assert len(diff_names_spills) == 3
        Expecting nothing
        **********************************************************************
        Line 1, in 7Ac 0
        Failed example:
            assert len(diff_names_spills) == 3
        Exception raised:
            Traceback (most recent call last):
              File "/opt/anaconda3/envs/eds220-env/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest 7Ac 0[0]>", line 1, in <module>
                assert len(diff_names_spills) == 3
                       ^^^^^^^^^^^^^^^^^^^^^^
            TypeError: object of type 'ellipsis' has no len()

    7Ac - 2 result:
        ❌ Test case failed
        Trying:
            assert isinstance(diff_names_spills, list)
        Expecting nothing
        ******************************************************************